In [ ]:
import os, torch

### load graph data 
import pickle

with open('../data/graphs/graph.pkl', 'rb') as f:
    data = pickle.load(f)


In [ ]:
### create edge sampling loader
import torch_geometric.transforms as T
import numpy as np
from torch_geometric.nn.models import Node2Vec

model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                    context_size=10, walks_per_node=10,
                    num_negative_samples=1, p=0.9, q=0.5, sparse=True)

loader = model.loader(batch_size=128, shuffle=True)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

for i in range(1, 101):
    loss = train()
    print('Epoch: {:02d}, Loss: {:.4f}'.format(i, loss))
